<a href="https://colab.research.google.com/github/alfcastillo90/pyspark_streaming_ubo/blob/main/pyspark_streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
## descargamos e instalamos jdk y apache spark
%%capture
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget 'https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz' -O 'spark-3.3.1-bin-hadoop3.tgz'
!tar -xvf /content/spark-3.3.1-bin-hadoop3.tgz
!pip install -q findspark 
!pip install background
import os 
import findspark 
## creamos las variables de entorno JAVA_HOME y SPARK_HOME
os.environ['JAVA_HOME']='/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME']='/content/spark-3.3.1-bin-hadoop3'

In [2]:
## inicializamos
findspark.init()

In [3]:
## nos integramos con google drive
from google.colab import drive 
drive.mount("/content/gdrive")
gdrive='content/gdrive/MyDrive/Estudios/UBO/BIGDATA'

Mounted at /content/gdrive


In [4]:
## creamos la carpeta streaming, primero validamos si existe o no
!test -d /content/streaming && echo 'la carpeta existe' || mkdir streaming

In [5]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.master('local').appName('PySparkStreaming').config('spark.ui.port',4050).getOrCreate()

In [6]:
## Definimos el esquema

from pyspark.sql.types import StructType, StructField, IntegerType, StringType

structureSchema = StructType([ \
                     StructField('index', IntegerType(), True), \
                     StructField('Order ID', StringType(), True), \
                     StructField('Date', StringType(), True), \
                     StructField('Status', StringType(), True), \
                     ])


In [7]:
## Leemos el archivo csv y le asignamos el esquema que fue definido previamente
dataFrame = spark.read.csv('/'+gdrive+'/amazon-orders-status.csv', sep=',', header=True, schema=structureSchema)


In [8]:
dataFrame.printSchema()

root
 |-- index: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Status: string (nullable = true)



In [9]:
streamingDataFrame = spark.readStream.format('csv').schema(structureSchema).option('header', True).load('streaming')

In [ ]:
streamingDataFrame.isStreaming

In [23]:
dataFrame.count()

128975

In [25]:
from background import task
from time import sleep
import pandas as pd 
import numpy as np


@task
def index(n):
  pd.DataFrame(dataFrame.filter((dataFrame['index'] >= n) & (dataFrame['index'] <= (n +10 ))).toPandas()).to_csv('/content/streaming/data'+ str(n)+'.csv' ,index=False)

def ciclo():
  for n in range(0,dataFrame.count()):
    index(n)

ciclo()

In [ ]:
result_df=streamingDataFrame.select('*')
query=(result_df.writeStream.format('json').queryName('selectTable')\
       .option('checkpointLocation','checkpoint')\
       .option('path','results')\
       .start()\
       .awaitTermination())